In [14]:
import MetaTrader5 as mt5
from backbone.utils.general_purpose import calculate_units_size

In [48]:
mt5.initialize()

info = mt5.symbol_info('BTCUSD')
info

SymbolInfo(custom=False, chart_mode=0, select=False, visible=False, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=0, digits=2, spread=0, spread_float=True, ticks_bookdepth=0, trade_calc_mode=4, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=0, trade_freeze_level=0, trade_exemode=2, swap_mode=5, swap_rollover3days=5, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=3, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=0.0, bidhigh=0.0, bidlow=0.0, ask=0.0, askhigh=0.0, asklow=0.0, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=0.01, trade_tick_value=0.01, trade_tick_value_profit=0.01, trade_tick_value_loss=0.01, trade_tick_size=0.01, trade_contract_size=1.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=0.01, volume_max=100.0, volume_step=0.01, volume_limit=0.0, swap_lo

In [51]:
info.swap_mode

5

In [16]:
contract_volume = info.trade_contract_size
minimum_lot = info.volume_min
maximum_lot = info.volume_max
pip_value = info.trade_tick_size
minimum_units = contract_volume * minimum_lot
trade_tick_value_loss = info.trade_tick_value_loss
volume_step = info.volume_step

print('contract_volume: ', contract_volume)
print('minimum_lot: ', minimum_lot)
print('maximum_lot: ', maximum_lot)
print('pip_value: ', pip_value)
print('minimum_units: ', minimum_units)
print('trade_tick_value_loss: ', trade_tick_value_loss)
print('volume_step: ', volume_step)

contract_volume:  100000.0
minimum_lot:  0.01
maximum_lot:  50.0
pip_value:  1e-05
minimum_units:  1000.0
trade_tick_value_loss:  1.0
volume_step:  0.01


In [7]:
from backbone.utils.general_purpose import diff_pips


pips = diff_pips(87.07, 89, pip_value=pip_value)

print('pips: ', pips)

lots = calculate_units_size(
    199_274, 
    1, 
    pips, 
    10_000, 
    1, 
    True, 
    contract_volume, 
    trade_tick_value_loss, 
    volume_step
)

print('lots: ', lots)

pips:  193.00000000000068
lots:  1032.0


In [29]:
import pandas as pd

strategy = 'BPercent'
ticker = 'EURUSD'
timeframe = '16386'

path = f'./backtesting_pipeline/strategies/{strategy}/preliminar_analysis/{ticker}_{timeframe}/trades.csv'
trades = pd.read_csv(path)

trades['Duration'] = pd.to_timedelta(trades['Duration'])
trades['Duration'] = trades['Duration'].dt.total_seconds() // 3600

print('duracion promedio: ', trades['Duration'].mean())
print('duracion desvio: ', trades['Duration'].std())
print('mediana duracion: ', trades['Duration'].median())

duracion promedio:  30.337349397590362
duracion desvio:  30.857355926098222
mediana duracion:  18.0


In [32]:
import numpy as np
trades['Size'] = np.abs(trades['Size']) / 100_000
trades['Size'].describe()

count    166.000000
mean       0.303855
std        0.111848
min        0.120000
25%        0.220000
50%        0.289990
75%        0.360000
max        0.680000
Name: Size, dtype: float64

In [33]:
trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,0.27000,311,317,1.205526,1.209156,-98.016149,-0.003011,2021-02-08 22:00:00+00:00,2021-02-09 10:00:00+00:00,12.0
1,0.28999,380,388,1.210864,1.207449,-99.032623,-0.002820,2021-02-16 16:00:00+00:00,2021-02-17 08:00:00+00:00,16.0
2,0.19000,474,477,1.212544,1.207470,-96.415686,-0.004185,2021-02-26 12:00:00+00:00,2021-02-26 18:00:00+00:00,6.0
3,0.26000,581,585,1.193796,1.197544,-97.453465,-0.003140,2021-03-11 10:00:00+00:00,2021-03-11 18:00:00+00:00,8.0
4,0.22000,586,605,1.197196,1.192010,114.093223,0.004332,2021-03-11 20:00:00+00:00,2021-03-15 10:00:00+00:00,86.0
...,...,...,...,...,...,...,...,...,...,...
161,0.25000,11673,11681,1.113252,1.108205,-126.180473,-0.004534,2024-09-30 22:00:00+00:00,2024-10-01 14:00:00+00:00,16.0
162,0.52000,11898,11900,1.083818,1.080510,172.032806,0.003052,2024-10-25 16:00:00+00:00,2024-10-25 20:00:00+00:00,4.0
163,0.48000,11911,11922,1.082388,1.077740,223.120886,0.004295,2024-10-28 18:00:00+00:00,2024-10-29 16:00:00+00:00,22.0
164,0.44000,11932,11935,1.083448,1.086378,-128.891112,-0.002704,2024-10-30 12:00:00+00:00,2024-10-30 18:00:00+00:00,6.0


1e-05